In [5]:
import cv2
import numpy as np
import sys

In [6]:
def resize(dst,img):
    width = img.shape[1]
    height = img.shape[0]
    dim = (width, height)
    resized = cv2.resize(dst, dim, interpolation = cv2.INTER_AREA)
    return resized

In [7]:
def genFrames():
    #capture video
    video = cv2.VideoCapture(0)
    bg = cv2.imread('background_1.jpg')
    success, ref_img = video.read()
    flag = 0
    #threshold vars
    diff_threshold = 30
    gray_threshold = 14

    #capture frame
    while(1):
            success, img = video.read()
            success2 = True
            bg = resize(bg,ref_img)
            if flag==0:
                    ref_img = img

            # create a mask
            diff1=cv2.subtract(img,ref_img)
            diff2=cv2.subtract(ref_img,img)
            diff = diff1+diff2
            diff[abs(diff)<diff_threshold]=0 ##what happens when we change this threshold (was 14) - 20 works well - 30 is better, try for 28
            gray = cv2.cvtColor(diff.astype(np.uint8), cv2.COLOR_BGR2GRAY)
            gray[np.abs(gray) < gray_threshold] = 0 #was 10 
            fgmask = gray.astype(np.uint8)
            fgmask[fgmask>0]=255

            #invert the mask
            fgmask_inv = cv2.bitwise_not(fgmask)

            #use the masks to extract the relevant parts from FG and BG
            fgimg = cv2.bitwise_and(img,img,mask = fgmask)
            bgimg = cv2.bitwise_and(bg,bg,mask = fgmask_inv)

            #combine both the BG and the FG images
            dst = cv2.add(bgimg,fgimg)

            #show images
            cv2.imshow('Background Removal',dst)
            key = cv2.waitKey(5) & 0xFF
            if ord('q') == key:
                    break
            elif ord('d') == key:
                    flag = 1
                    print("Background Captured")
            elif ord('r') == key:
                    flag = 0
                    print("Ready to Capture new Background")

    video.release()
    cv2.destroyAllWindows()

    #return jpeg.tobytes()

In [8]:
genFrames()